<a href="https://colab.research.google.com/github/ipavlopoulos/ppast-gr/blob/main/download_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PDF SCRAPER

In [ ]:
import os
from selenium import webdriver
from time import sleep
import time
from tqdm import tqdm
from PyPDF2 import PdfFileReader, PdfFileWriter
import re

In [ ]:
"""
Change parameters:
var = set to the number of files existing
endings[_:_] = from number of files existing + 3
if int(num[0]) > __ = greater than number of files existing

"""
def pdf_scraper(index, download_folder_path):
  failed_access = []
  crd_list = []

  first = "https://digitallib.parliament.gr/main.asp?current="
  endings = [i for i in range(12256712, 12284152)] #the interval of interest
  var = index   #set var equal to the number of files already existing
  dir_check = 0
  crd = 0

  for i in tqdm(endings[var+3:var+100]):

    url = first + str(i)
    print(url)
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    download_folder = download_folder_path#"/content/drive/MyDrive/Downloads_Colab"
    profile = {"plugins.plugins_list": [{"enabled": False,
                                        "name": "Chrome PDF Viewer"}],
            "download.default_directory": download_folder,
            "download.extensions_to_open": "",
            "plugins.always_open_pdf_externally": True}
    options.add_experimental_option("prefs", profile)

    driver = webdriver.Chrome('chromedriver',chrome_options = options)
    #IMPORTANT
    driver.get(url)
    dir_check+=1
    time.sleep(4)
    driver.close()
    #IF FAILED ACCESS
    if len(os.listdir(download_folder)) <dir_check:
      failed_access.append(url)
      print(f'Number of files failed: {len(failed_access)}\nItems in dir: {len(os.listdir(download_folder))}\n.')
      var+=1
      dir_check-=1

    for file in os.listdir(download_folder):

      if file == "main.pdf":
        file = download_folder + "/" + file
        var += 1
        name = download_folder + "/" + "main" + str(var) + ".pdf"
        os.rename(file, name)
        print(f'Folder Done\n\nN. {i}\n\nFile : {var}\nFiles in dir: {len(os.listdir(download_folder))}')
    #IF CORRUPTED FILE
    if 'main.pdf.crdownload' in os.listdir(download_folder):
        var+=1
        print(f'.\nWARNING!!! crdownload for file:\nFile: {url}\n')
        crd_list.append(url)
        os.remove('/content/drive/MyDrive/Downloads_Colab/main.pdf.crdownload')

# Cropping

In [ ]:

def pdf_horizontal_crop(download_folder):
  for pdf in os.listdir(download_folder):# folder with PDF files
    num = re.findall(r'\d+', pdf )
    if int(num[0]) > 0: #set int(num[0]) greater than the existing files.

      cur_file = download_folder + "\\" + pdf
      reader = PdfFileReader(cur_file)
      writer = PdfFileWriter()

      for page in reader.pages:
        page.cropbox.lower_left = ((10,362))
        page.cropbox.upper_right = ((602,782))
        writer.add_page(page)

      with open(cur_file,'wb') as fp:
        writer.write(fp)

def pdf_vertical_crop(download_folder, output_folder):
  for pdf in tqdm(os.listdir(download_folder)):# folder with PDF files
    time.sleep(0.2)
    if 'main' in pdf:

      num = re.findall(r'\d+', pdf )
      cur_file = download_folder + '\\' + pdf
      reader = PdfFileReader(cur_file, 'r')
      writer = PdfFileWriter()
      for page in reader.pages:

        page.cropbox.lower_left = ((10,362))
        page.cropbox.upper_right = ((301,782))
        half_file = output_folder + '\\' + 'main' + str(num[0]) + '_1' + '.pdf'
        writer.add_page(page)
        with open(half_file, 'wb+') as half_pdf:
          writer.write(half_pdf)

        page.cropbox.lower_left = ((301,362))
        page.cropbox.upper_right = ((610,782))
        half_file = output_folder + '\\' + 'main' + str(num[0]) + '_2' + '.pdf'
        writer.add_page(page)
        with open(half_file, 'wb+') as half_pdf:
          writer.write(half_pdf)

# JPG Extraction

In [ ]:
import fitz
import io
from PIL import Image
import os
from time import sleep
from tqdm import tqdm

In [ ]:
def jpg_extraction(input_folder, output_folder):

  list_dir = os.listdir(input_folder)
  for i in tqdm(list_dir):
    name = i.replace('main','').replace('.pdf','')
    file = input_folder + '/' + i
    doc = fitz.open(file)

    for page in doc:
      pix = page.get_pixmap(matrix=fitz.Identity, dpi=300,colorspace=fitz.csRGB, clip=None, alpha=True, annots=True)
      pix.save(f"{output_folder}/main{name}.jpg")  # save file
    sleep(0.02)

# XML Parser

In [ ]:
import xml.etree.ElementTree as ET
from xml.dom.minidom import parseString
import os
import re
from tqdm import tqdm
import pandas as pd
from natsort import natsorted

In [ ]:
def xml_parser(xml_path, txt_path):

  dir = os.listdir(xml_path)
  for file in tqdm(dir): #accessing xml file
    xml_file = xml_path + '/' + file
    mytree = ET.parse(xml_path + '/' + file)
    myroot = mytree.getroot()
    with open(xml_file,'r') as xml:
      data = xml.read()
      dom = parseString(data)
      textRegion = len(dom.getElementsByTagName('TextRegion')) #counting the text regions

    txt_file = txt_path + '/' + file.replace('xml', 'txt')  #creating the txt file of the page
    f = open(txt_file,'w')
    count_Region = 0
    count_Line = 1


    while count_Region < textRegion:
      count_Region += 1
      textLine_list = [i for i in myroot[1][count_Region] if i.tag[-8:] == 'TextLine'] #counting lines per region
      while count_Line < len(textLine_list) + 1:
        line = str(myroot[1][count_Region][count_Line][-1][0].text)
        f.write(f'{line} ')

        count_Line+=1
      count_Line = 1
    f.close

# Create Dataset

In [ ]:
import os
import re
from tqdm import tqdm
import pandas as pd
from natsort import natsorted

In [ ]:
"""
Create three lists
Text_list = list with text from each page
year_list = list of the period (found in the website of the Greek Parliament)
committee_list = list of the committee (found in the website of the Greek Parliament)
Combine the lists to one dataframe
"""
def csv_creator(txt_path):
  text_list = [] #list with text files
  year_list = [] # list with period
  committee_list = [] # list of committee
  for i in natsorted(os.listdir(txt_path)):
    num = re.findall(r'\d+', i)
    txt = txt_path + '/' + i
    # add text to list
    with open(txt, 'r') as f:
      text = f.read()
      f.close()
    text_list.append(text)

    # add period and committee to list based on id
    if int(num[0]) <= 141:
      year_list.append('13/05/1946 - 20/06/1946')
      committee_list.append("ΑΝΑΘΕΩΡΗΤΙΚΗ - ΣΥΝΟΔΟΣ Α' (T.1)")
    elif 141 < int(num[0]) <= 306:
      year_list.append('21/06/1946 - 31/07/1946')
      committee_list.append("ΑΝΑΘΕΩΡΗΤΙΚΗ - ΣΥΝΟΔΟΣ Α' (T.2)")
    elif 306 < int(num[0]) <= 545:
      year_list.append('01/10/1946 - 27/02/1947')
      committee_list.append("ΑΝΑΘΕΩΡΗΤΙΚΗ - ΣΥΝΟΔΟΣ Α' (T.3)")
    elif 545 < int(num[0]) <= 802:
      year_list.append('21/04/1947 - 13/09/1947')
      committee_list.append("ΑΝΑΘΕΩΡΗΤΙΚΗ - ΣΥΝΟΔΟΣ Α' (T.4)")
    elif 802 < int(num[0]) <= 1093:
      year_list.append('09/08/1946 - 18/04/1947')
      committee_list.append("ΑΝΑΘΕΩΡΗΤΙΚΗ - ΣΥΝΟΔΟΣ Α' - ΕΠΙΤΡΟΠΗ (T.1)")
    elif 1093 < int(num[0]) <= 1319:
      year_list.append('08/07/1947 - 31/10/1947')
      committee_list.append("ΑΝΑΘΕΩΡΗΤΙΚΗ - ΣΥΝΟΔΟΣ Α' - ΕΠΙΤΡΟΠΗ (T.2)")
    elif 1319 < int(num[0]) <= 1589:
      year_list.append('01/11/1947 - 23/01/1948')
      committee_list.append("ΑΝΑΘΕΩΡΗΤΙΚΗ - ΣΥΝΟΔΟΣ Β' (T.1)")

  columns = ['id', "period", 'comitee', 'text']
  minutes_df = pd.DataFrame({'id': id_list,
                              'period' : year_list,
                              'commitee' : committee_list,
                              'text': text_list},
                              columns = columns)

  return minutes_df

# RegEx

In [ ]:
import pandas as pd
import os
import re
from natsort import natsorted
from tqdm import tqdm

In [ ]:
def regex_check(csv_file_path):
  minutes_df = pd.read_csv(csv_file_path) #path of csv with image id, period, committee and text

  #add columns for the data extracted by RegEx
  minutes_df['Date'] = ''
  minutes_df['Orator'] = ''
  minutes_df['Applause'] = ''
  i = 0
  for row in minutes_df.text:
    if isinstance(row, float) == False:
      date = re.findall(r'([Α-ΩΆΈΎΊΌΏἈἘἸὈὨᾺῈῪῚῸῺἊἚἺὊὪα-ωάέύίήόὶὺὰὲὸὴῖῦᾶῆ]+\s+[0-9]+\s+[Α-ΩΆΈΎΊΌΏἈἘἸὈὨᾺῈῪῚῸῺἊἚἺὊὪα-ωάέύίήώόὶὺὰὲὼὸῖῦᾶῆϊϋΐΰ]+\s+[0-9]+\.*)', row) #RegEx for date
      if date:
          if date[0] in row:
            minutes_df.loc[i, 'Date'] = date[0]
    i += 1

  rows = 0
  for text in tqdm(minutes_df.text.tolist()):
    orators = []
    if isinstance(text, float) == False:
      orator = re.findall(r"[\s\.]*([Α-ΩἈἉἊἘἙἚἸἹἺὈὉὊ'ΥὙὛὨὩὫ]\s([α-ωάᾶὰἆέὲύϋῦὺὖὒίϊῖὶἶόὸώῶὼὦὢ]{1,3}\s){1,8})[\s\.]*", text) #RegEx for Orator
      if len(orator) >= 1:
        orators.append(orator[0])

    minutes_df.loc[rows, 'Orator'] = ','.join(','.join(tup) for tup in orators)
    rows += 1

  rows = 0
  for text in tqdm(minutes_df.text.tolist()):
    claps = []
    if isinstance(text, float) == False:
      clap = re.findall(r"(\(.*(κρο).*\))", text) #RegEx for Applause
      if 15 > len(clap) >= 1:
        claps.append(clap[0])

    minutes_df.loc[rows, 'Applause'] = ','.join(','.join(tup) for tup in claps)
    rows += 1